In [1]:
import cv2
import numpy as np

def create_circle_image(image_size=(256, 256), circle_count=5, min_radius=10, max_radius=50):
    # Create a blank image with a random background color
    background_color = np.random.randint(0, 256, size=(3,), dtype=np.uint8)
    image = np.full((image_size[0], image_size[1], 3), background_color, dtype=np.uint8)

    # List to store the circle coordinates and radii for annotation
    circles = []

    for _ in range(circle_count):
        # Random center, radius, and color for each circle
        center = (np.random.randint(0, image_size[1]), np.random.randint(0, image_size[0]))
        radius = np.random.randint(min_radius, max_radius)
        color = np.random.randint(0, 256, size=(3,), dtype=np.uint8).tolist()

        # Draw the circle on the image
        cv2.circle(image, center, radius, color, -1)  # -1 fill the circle

        # Store the circle parameters for annotation (YOLO format: [class, x_center, y_center, width, height])
        x_center = center[0] / image_size[1]
        y_center = center[1] / image_size[0]
        width = (2 * radius) / image_size[1]
        height = (2 * radius) / image_size[0]
        circles.append([0, x_center, y_center, width, height])

    return image, circles


In [2]:
import os

def generate_dataset(image_count=1000, image_size=(256, 256), save_dir='circle_dataset'):
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(f"{save_dir}/images", exist_ok=True)
    os.makedirs(f"{save_dir}/labels", exist_ok=True)

    for i in range(image_count):
        if i % 2 == 0:  # Half of the images will contain circles
            image, circles = create_circle_image(image_size=image_size, circle_count=np.random.randint(1, 6))
        else:  # Half will be negative examples without circles
            image = np.full((image_size[0], image_size[1], 3), np.random.randint(0, 256, size=(3,), dtype=np.uint8), dtype=np.uint8)
            circles = []

        # Save the image
        image_path = f"{save_dir}/images/image_{i:04d}.jpg"
        cv2.imwrite(image_path, image)

        # Save the annotations in YOLO format
        label_path = f"{save_dir}/labels/image_{i:04d}.txt"
        with open(label_path, 'w') as f:
            for circle in circles:
                f.write(" ".join(map(str, circle)) + "\n")

    print(f"Dataset generated with {image_count} images in '{save_dir}'.")

# Generate the dataset
generate_dataset(image_count=1000, image_size=(256, 256))


Dataset generated with 1000 images in 'circle_dataset'.


In [1]:
# Clone the YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

# Install required dependencies
!pip install -r requirements.txt


fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split

def prepare_yolo_dataset(original_dataset_dir='/content/circle_dataset', yolo_dataset_dir='yolo_circle_dataset', val_split=0.2):
    # Create the YOLO dataset structure
    os.makedirs(f'{yolo_dataset_dir}/images/train', exist_ok=True)
    os.makedirs(f'{yolo_dataset_dir}/images/val', exist_ok=True)
    os.makedirs(f'{yolo_dataset_dir}/labels/train', exist_ok=True)
    os.makedirs(f'{yolo_dataset_dir}/labels/val', exist_ok=True)

    # Get a list of images and split them into training and validation sets
    images = os.listdir(f'{original_dataset_dir}/images')
    train_images, val_images = train_test_split(images, test_size=val_split)

    def move_files(image_list, subset):
        for image in image_list:
            # Move image
            shutil.move(f'{original_dataset_dir}/images/{image}', f'{yolo_dataset_dir}/images/{subset}/{image}')
            # Move corresponding label
            label = image.replace('.jpg', '.txt')
            shutil.move(f'{original_dataset_dir}/labels/{label}', f'{yolo_dataset_dir}/labels/{subset}/{label}')

    # Move the files to the respective directories
    move_files(train_images, 'train')
    move_files(val_images, 'val')

    print(f"Dataset prepared for YOLO with {len(train_images)} training images and {len(val_images)} validation images.")

# Prepare the YOLO dataset
prepare_yolo_dataset()


Dataset prepared for YOLO with 800 training images and 200 validation images.


In [8]:
train = "../yolo_circle_dataset/images/train"
val = "../yolo_circle_dataset/images/val"
nc: 1  # number of classes (only circles in this case)
names: ['circle']  # class names


In [10]:
!ls /content/yolov5/


benchmarks.py	 data	     LICENSE	     README.md	       train.py        yolo_circle_dataset
CITATION.cff	 detect.py   models	     README.zh-CN.md   tutorial.ipynb
classify	 export.py   __pycache__     requirements.txt  utils
CONTRIBUTING.md  hubconf.py  pyproject.toml  segment	       val.py


In [11]:
data_yaml_content = """
train: ../yolo_circle_dataset/images/train
val: ../yolo_circle_dataset/images/val

nc: 1  # number of classes (only circles in this case)
names: ['circle']  # class names
"""

with open('/content/yolov5/data.yaml', 'w') as f:
    f.write(data_yaml_content)


In [12]:
!python train.py --img 256 --batch 16 --epochs 30 --data data.yaml --cfg models/yolov5s.yaml --weights yolov5s.pt --name circle_detection


2024-08-10 14:42:37.702902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-10 14:42:37.723394: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-10 14:42:37.729763: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=models/yolov5s.yaml, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=256, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, sing

In [13]:
# Evaluate the model
!python val.py --weights runs/train/circle_detection/weights/best.pt --data data.yaml --img 256


val: data=data.yaml, weights=['runs/train/circle_detection/weights/best.pt'], batch_size=32, imgsz=256, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-350-g6096750f Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/yolov5/yolo_circle_dataset/labels/val.cache... 200 images, 111 backgrounds, 0 corrupt: 100% 200/200 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 7/7 [00:03<00:00,  1.90it/s]
                   all        200        281      0.985      0.986      0.994       0.95
Speed: 0.0ms pre-process, 3.2ms inference, 6.7ms NMS per image at shape (32, 3, 256, 256)
Results save